In [2]:
import pandas as pd
import numpy as np
import glob
import os
import geopandas as gpd
import urllib
import sys
import pathlib
import glob
import matplotlib.pyplot as plt
from urllib.parse import quote
from sqlalchemy import create_engine
import configparser


import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px
from fluxdataqaqc import Data, QaQc, Plot
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import LinearAxis, Range1d
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
sys.path.append("//")
#sys.path.append("../../Micromet")
import micromet
from micromet import AmerifluxDataProcessor
%matplotlib inline

In [4]:
from pyproj import Transformer
EPSG = 5070
# load initial flux data
station = 'US-UTW'
config_path = f'../../station_config/{station}.ini'
config = configparser.ConfigParser()
config.read(config_path)

latitude = config['METADATA']['station_latitude']
longitude = config['METADATA']['station_longitude']

# Define the transformer from WGS84 to NAD83 Conus Albers
transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)

# Perform the transformation
station_x, station_y = transformer.transform(longitude, latitude)

print(f"Projected Coordinates in NAD83 Conus Albers (EPSG:5070): X={station_x}, Y={station_y}")

Projected Coordinates in NAD83 Conus Albers (EPSG:5070): X=-1251001.2763440341, Y=1921956.3381891234


In [11]:
import math

def polar_to_cartesian_dataframe(df, wd_column='WD', dist_column='Dist'):
    """
    Convert polar coordinates from a DataFrame to Cartesian coordinates.

    Parameters:
        df (pd.DataFrame): Input DataFrame containing polar coordinates.
        wd_column (str): Column name for degrees from north.
        dist_column (str): Column name for distance from origin.

    Returns:
        pd.DataFrame: A DataFrame with added 'X' and 'Y' columns.
    """
    # Create copies of the input columns to avoid modifying original data
    wd = df[wd_column].copy()
    dist = df[dist_column].copy()

    # Identify invalid values (-9999 or NaN)
    invalid_mask = (wd == -9999) | (dist == -9999) | wd.isna() | dist.isna()

    # Convert degrees from north to standard polar angle (radians) where valid
    theta_radians = np.radians(90 - wd)

    # Calculate Cartesian coordinates, setting invalid values to NaN
    df[f'X_{dist_column}'] = np.where(invalid_mask, np.nan, dist * np.cos(theta_radians))
    df[f'Y_{dist_column}'] = np.where(invalid_mask, np.nan, dist * np.sin(theta_radians))

    return df


In [12]:
#df = pd.read_csv(config['METADATA']['climate_file_path'],skiprows=config['METADATA']['skiprows'])
df = pd.read_csv("../../station_data/US-CdM_HH_202306141100_202410081700.csv")
for col in df.columns:
    if "fetch" in col.lower():
        df = polar_to_cartesian_dataframe(df, wd_column='WD',dist_column=col)

In [13]:
df

,datetime_start,TIMESTAMP_START,TIMESTAMP_END,CO2,CO2_SIGMA,H2O,H2O_SIGMA,FC,FC_SSITC_TEST,LE,...,TS_2_1_1,SWC_2_1_1,X_FETCH_MAX,Y_FETCH_MAX,X_FETCH_90,Y_FETCH_90,X_FETCH_55,Y_FETCH_55,X_FETCH_40,Y_FETCH_40
0,2023-06-14 11:00:00,202306141100,202306141130,-9999.0000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.000000,...,-9999.00000,-9999.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-06-14 11:30:00,202306141130,202306141200,-9999.0000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.000000,...,-9999.00000,-9999.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-06-14 12:00:00,202306141200,202306141230,404.8652,7.160377,11.003180,1.408956,-9999.000000,-9999.0,394.658600,...,-9999.00000,-9999.000000,-22.087887,-0.541745,-60.935784,-1.494559,-31.392359,-0.769954,-23.760674,-0.582773
3,2023-06-14 12:30:00,202306141230,202306141300,403.2438,6.578680,10.929970,1.396008,-19.724970,-9999.0,277.552900,...,-9999.00000,-9999.000000,27.341082,-12.220065,75.411491,-33.705078,38.863654,-17.370065,29.415165,-13.147074
4,2023-06-14 13:00:00,202306141300,202306141330,397.2703,7.669390,12.208020,1.357026,-29.465460,-9999.0,290.635000,...,-9999.00000,-9999.000000,-12.573416,-13.561474,-35.182653,-37.947417,-17.948699,-19.359165,-13.577773,-14.644758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23143,2024-10-08 14:30:00,202410081430,202410081500,401.5762,0.373631,2.863288,0.053154,-0.284607,-9999.0,5.291050,...,25.01365,2.415389,12.467820,-90.431495,34.685692,-251.581992,17.772194,-128.905138,13.446685,-97.531394
23144,2024-10-08 15:00:00,202410081500,202410081530,401.5929,0.473523,2.466282,0.107060,-0.755953,2.0,0.461339,...,24.36275,2.412944,-77.916613,0.484539,-216.863007,1.348603,-111.061753,0.690658,-84.031255,0.522564
23145,2024-10-08 15:30:00,202410081530,202410081600,401.3630,0.335989,2.445982,0.044886,-0.116504,-9999.0,4.989396,...,23.67358,2.412833,-82.222816,25.811203,-228.782695,71.818954,-117.203015,36.792109,-88.677530,27.837453
23146,2024-10-08 16:00:00,202410081600,202410081630,401.1919,0.380539,2.550954,0.069491,-0.942505,2.0,11.713800,...,23.25630,2.413972,-72.149224,-11.916283,-200.821981,-33.168085,-102.834360,-16.984290,-77.806878,-12.850711
